In [2]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    temperature=0, 
    # groq_api_key='', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.atlassian.com/company/careers/details/17052")
page_data = loader.load().pop().page_content
print(page_data)


         Job Details | Atlassian                                  Close   View this page in your language?   All languages   Choose your language    中文   Deutsch   English   Español   Français   Italiano   한국어   Nederlands   日本語   Português   Pусский   Polski                            Get it free     Products    Featured    Developers    Product Managers    IT professionals    Business Teams    Leadership Teams       Featured   Developers   Product Managers   IT professionals   Business Teams   Leadership Teams    See all products       Featured            Jira  Flexible project management             Confluence  Knowledge, all in one place             Jira Service Management  High-velocity service delivery             Trello  Organized & visualized work             Rovo NEW  Unlock enterprise knowledge             Jira Product Discovery NEW  Capture & prioritize ideas             Compass NEW  Optimize software health             Guard NEW  Enhanced cloud security             Loom NE

In [54]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `all skills with proper bullet points` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

type(res.content)

```json
{
  "role": "Business Intelligence Analyst",
  "experience": "2+ Years of Experience in Business Intelligence or related fields",
  "skills": [
    "Bachelor’s degree or equivalent in a STEM field (e.g. statistics, mathematics, physics, econometrics, or computer science)",
    "Proficiency in SQL",
    "Experience telling stories with data and familiarity with Tableau",
    "Experience working cross-functionally to collect requirements and feedback and manage a transparent backlog",
    "Experience applying your analytics skills to projects which have had impact on business strategy and performance",
    "Ability to craft analysis into well-written content"
  ],
  "description": "Collaborate on a variety of product and business problems with a diverse set of cross-functional partners and become a trusted strategic partner through the structure and clarity of your work. Apply technical expertise with BI tools to create simple to understand data visualizations. Partner with Data 

str

In [55]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
# json_res['job_postings'][0]

In [56]:
type(json_res)

dict

#  Now we are set up the Chroma DB..

In [57]:
import pandas as pd
dataset = pd.read_csv("my_portfolio.csv")
dataset

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [58]:
#  We need to store this data into the chromaDb. This will create the vectorstore
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in dataset.iterrows():
        collection.add(documents=row["Techstack"],metadatas={"links": row["Links"]},ids=[str(uuid.uuid4())])

In [61]:
links = collection.query(query_texts= job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/devops-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/kotlin-backend-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/angular-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}]]

In [60]:
job = json_res


In [62]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Business Intelligence Solutions for Enhanced Decision-Making

Dear Hiring Manager,

I came across the Business Intelligence Analyst role at your esteemed organization and was impressed by the job description. As a Business Development Executive at AtliQ, I believe our company can fulfill your needs and provide tailored solutions to drive business growth.

At AtliQ, we have a proven track record of empowering enterprises with scalable, optimized, and efficient solutions. Our team of experts can collaborate with your cross-functional partners to structure and clarify work, applying technical expertise with BI tools to create simple-to-understand data visualizations. We can also partner with your Data Engineers and Data Scientists to define and govern key business metrics, informing and influencing strategy decisions.

Our expertise in data analysis, visualization, and interpretation can help identify data quality issues in real-time, proactively resolving them with engine